In [2]:
import heapq
import math
import numpy as np
import time

def verify_input(initial_state, n):
    # initial error = 0
    err = 0 
    reason = "Input is valid"

    if n < 2 or n >= 128:
        err = -1
        reason = "n is not between 2 and 128"

    if len(initial_state) != n ** 2:
        err = -1
        reason = "Board size is not N^2"

    sorted_initial_state = sorted(initial_state)
    valid_list = range(n**2)

    if sorted_initial_state != valid_list:
        err = -1
        reason = "Your range of numbers do not match the number of tiles"

      if err == -1:
          raise ValueError(reason)
    return err, initial_state, reason
  
def solvePuzzle(n, initial_state, heuristic, prnt = False):
    start_time = time.time()
    heap = []
    frontierSize = 0
    link = {} # link of parent node
    heuristic_value = {} # cache heuristic function
    visited = {} # shortest path to a node
    list_of_lists_solution = [] # lists of lists solution if prnt = True

    # verifying input and raise error if invalid
    err, initial_state, reason = verify_input(initial_state, n)
    
    # goal state
    finish = PuzzleNode(n)
    
    # initialize board with the given input 
    # print the initial board
    p = PuzzleNode(n, current_state = initial_state)
    print "Initial board:"
    print p.__str__()
    print ""
    
    visited[p] = 0
    heuristic_value[p] = 0
    link[p] = None
    
    if heuristic == manhattanDistance:
      print "Solving using Manhattan Distance..."
    elif heuristics == misplacedTiles:
      print "Solving using Misplaced Tiles..."
      
    print ""
    
    # using heapq as a priority queue
    heapq.heappush(heap, (0, 0, p))
    
    # keep a count of the number of steps with a finite loop to ensure termination
    for steps in xrange(1000000):
        f, junk, current = heapq.heappop(heap)
        if current == finish:
            print "Final board:"
            print current
            
            if prnt == True:
                print ""
                print "Solutions as lists of lists:"
                print list_of_lists_solution
                print ""
            runtime = time.time() - start_time
            return steps, frontierSize, err, runtime

        # using the get_moves() method to get a list of moves for the current state  
        moves = current.get_moves()
        
        # saving shortest distance
        distance = visited[current]
        
        for mv in moves:
            if prnt == True:
                list_of_lists_solution.append(mv.current_state)

            # check for repeated states and avoid expanding them
            if mv not in visited or visited[mv] > distance + 1:
                visited[mv] = distance + 1
                if mv not in heuristic_value:
                    heuristic_value[mv] = heuristic(mv)
                    
                link[mv] = current
                heapq.heappush(heap, (visited[mv] + heuristic_value[mv], -steps, mv))
                
        frontierSize = max(frontierSize,len(heap)) 
    else:
        raise Exception("Did not find a solution")

class PuzzleNode(object):
    def __init__(self, n, current_state=None):
        # create an nxn block puzzle
        # with current_state as a specific state
        self.n = n
        self.n2 = n * n
        if current_state is None:
            self.current_state = [(x) % self.n2 for x in xrange(self.n2)]
        else:
            self.current_state = list(current_state)
        self.hsh = None
        self.last_move = []

    def __hash__(self):
        if self.hsh is None:
            self.hsh = hash(tuple(self.current_state))
        return self.hsh

    def __repr__(self):
        return "PuzzleNode(%d, %s)" % (self.n, self.current_state)

    def __str__(self):
        # print method to show grid of board state
        ys = ["%2d" % x for x in self.current_state]
        current_state = [" ".join(ys[steps:steps+self.n]) for steps in xrange(0,self.n2, self.n)]
        return "\n".join(current_state)

    def __eq__(self, other):
        return self.current_state == other.current_state

    def copy(self):
        return PuzzleNode(self.n, self.current_state)

    def get_moves(self):
        # find the 0 tile, and then generate any moves we
        # can by sliding another block into its place.
        tile0 = self.current_state.index(0)
        def swap(i):
            j = tile0
            tmp = list(self.current_state)
            last_move = tmp[i]
            tmp[i], tmp[j] = tmp[j], tmp[i]
            result = PuzzleNode(self.n, tmp)
            result.last_move = last_move
            return result

        if tile0 - self.n >= 0:
            yield swap(tile0 - self.n)
        if tile0 + self.n < self.n2:
            yield swap(tile0 + self.n)
        if tile0 % self.n > 0:
            yield swap(tile0 - 1)
        if tile0 % self.n < self.n - 1:
            yield swap(tile0 + 1)
            
    def build_path(state, finish, parent):
        # construct the path from state to finish given
        # a dict of parent links.

        x = finish
        current_state = [x]
        while x != state:
            x = parent[x]
            current_state.append(x)
        current_state.reverse()
        return current_state
      
def misplacedTiles(position):
    # heuristic returns number of misplaced tile
    n2 = position.n2
    c = 0
    for steps in xrange(n2):
        if position.current_state[steps] != steps:
            c += 1
    return c
  
def manhattanDistance(position):
    # heuristic returns distance between tiles to the goal state
    n2 = position.n2
    state = []
    for i in xrange(n2):
        state.append(position.current_state[i])
    index = [value for value in xrange(n2)]
    
    manhattan = 0
    for var in xrange(n2):
        idx = state.index(var)
        board_x = idx / 3
        board_y = idx % 3
        x = index[var] / 3
        y = index[var] % 3
        manhattan += math.fabs(x - board_x)
        manhattan += math.fabs(y - board_y)
    
        return manhattan
         
heuristics = [misplacedTiles, manhattanDistance]
  
def main(n, initial_state):
    # first, flatten the input board to a 1-D array
    initial_state = [item for sublist in initial_state for item in sublist]
    # solving p (n * n) to goal state using heuristic
    steps, frontier_size, err, runtime = solvePuzzle(n, initial_state, heuristics[0])
    print "Number of steps:", steps
    print "Frontier size:", frontier_size
    print "Error:", err
    print "Runtime: {}s".format(runtime)

n = 3
current_state = [[2, 3, 7], [1, 8, 0], [6, 5, 4]] 
main(n, current_state)

Initial board:
 2  3  7
 1  8  0
 6  5  4


Final board:
 0  1  2
 3  4  5
 6  7  8
Number of steps: 596
Frontier size: 356
Error: 0
Runtime: 0.011561870575s


In [4]:
def test_heuristics(n):
    for test_board in test_list: 
        test_item = [item for sublist in test_board for item in sublist]
        print("\nTesting Heuristics for board: {}".format(test_board))
        # misplacedTiles
        steps0, frontierSize0, err0, runtime0 = solvePuzzle(n = n, initial_state = test_item, heuristic = heuristics[0])
        # manhattanDist
        steps1, frontierSize1, err1, runtime1 = solvePuzzle(n = n, initial_state = test_item, heuristic = heuristics[1])

        print("\t        Misplaced Tiles  vs  Manhattan Distance ")
        print("Steps:          \t    {} \t \t   {}".format(steps0,steps1))
        print("Frontier size:  \t    {} \t  {}".format(frontierSize0,frontierSize1))
        print("Runtime (sec):\t       {0:10.3f}    {0:10.3f}".format(runtime0,runtime1))

# Test cases
test1 = [[5,7,6],[2,4,3],[8,1,0]]
test2 = [[7,0,8],[4,6,1],[5,3,2]]
test3 = [[2,3,7],[1,8,0],[6,5,4]]

test_list = [test1,test2,test3]

test_heuristics(n = 3)


Testing Heuristics for board: [[5, 7, 6], [2, 4, 3], [8, 1, 0]]
Initial board:
 5  7  6
 2  4  3
 8  1  0


Final board:
 0  1  2
 3  4  5
 6  7  8
Initial board:
 5  7  6
 2  4  3
 8  1  0

Solving using Manhattan Distance...

Final board:
 0  1  2
 3  4  5
 6  7  8
	        Misplaced Tiles  vs  Manhattan Distance 
Steps:          	    58958 	 	   160572
Frontier size:  	    20873 	  30576
Runtime (sec):	            1.344         1.344

Testing Heuristics for board: [[7, 0, 8], [4, 6, 1], [5, 3, 2]]
Initial board:
 7  0  8
 4  6  1
 5  3  2


Final board:
 0  1  2
 3  4  5
 6  7  8
Initial board:
 7  0  8
 4  6  1
 5  3  2

Solving using Manhattan Distance...

Final board:
 0  1  2
 3  4  5
 6  7  8
	        Misplaced Tiles  vs  Manhattan Distance 
Steps:          	    22171 	 	   97875
Frontier size:  	    10752 	  30141
Runtime (sec):	            0.509         0.509

Testing Heuristics for board: [[2, 3, 7], [1, 8, 0], [6, 5, 4]]
Initial board:
 2  3  7
 1  8  0
 6  5  4


Final bo

**COMPARISON OF THE TWO HEURISTIC FUNCTIONS **

```
Testing Heuristics for board: [[5, 7, 6], [2, 4, 3], [8, 1, 0]]
	        Misplaced Tiles  vs  Manhattan Distance 
Steps:          	    58961 	   159351
Frontier size:  	    20870 	    30803
Runtime (sec):	      1.358         1.358

Testing Heuristics for board: [[7, 0, 8], [4, 6, 1], [5, 3, 2]]
	        Misplaced Tiles  vs  Manhattan Distance 
Steps:          	    22167 	     97576
Frontier size:  	    10755 	     30082
Runtime (sec):	      0.520          0.520

Testing Heuristics for board: [[2, 3, 7], [1, 8, 0], [6, 5, 4]]
	        Misplaced Tiles  vs  Manhattan Distance 
Steps:          	    597 	 	   10977
Frontier size:  	    355 	         6016
Runtime (sec):	     0.009            0.009
```



In [5]:
test4 = [[1,0,3],[2,4,5],[6,7,8]] # unsolvable
test5 = [[7,0,2],[8,5,3],[6,4,1]] # unsolvable

solvable_or_not = [test1, test2, test3, test4, test5]

def isSolvable(n, board):
    print("\nTesting if board: {} is solvable".format(board))
    if type(board[1]) != int:
        board = [item for sublist in board for item in sublist] 
    
    inversions = 0 
    for i in range(0, n**2 -1): 
        if board[i] != 0: 
            for j in range(i, n**2): 
                if (board[i] > board[j]) and board[j] != 0: 
                    inversions += 1

    print "Total inversions:", inversions
    inversions_even = (inversions % 2 == 0) # Boolean value
    
    # If grid width is odd, then even number of inversions is solvable
    if n % 2 != 0:
        solvable = inversions_even 
    
    # When n is even: solvable situations are when row with blank = even & inversions = odd, 
    # OR row with blank = odd & inversions = even
    if n % 2 == 0:
        index_0 = board.index(0)
        row0 = index_0 / n
        row0_odd = row0 % 2 
        solvable =  (row0_odd == inversions_even)
        
    return solvable
  
for test_board in solvable_or_not:
    print isSolvable(3, test_board)


Testing if board: [[5, 7, 6], [2, 4, 3], [8, 1, 0]] is solvable
Total inversions: 18
True

Testing if board: [[7, 0, 8], [4, 6, 1], [5, 3, 2]] is solvable
Total inversions: 22
True

Testing if board: [[2, 3, 7], [1, 8, 0], [6, 5, 4]] is solvable
Total inversions: 12
True

Testing if board: [[1, 0, 3], [2, 4, 5], [6, 7, 8]] is solvable
Total inversions: 1
False

Testing if board: [[7, 0, 2], [8, 5, 3], [6, 4, 1]] is solvable
Total inversions: 19
False


In [6]:
# integrating isSolvable() into solvePuzzle() function

def solvePuzzle_solvable(n, initial_state, heuristic, prnt = False):
    start_time = time.time()
    
    heap = []
    frontierSize = 0
    link = {} # parent node link
    heuristic_value = {} # heuristic function cache
    visited = {} # shortest path to a node

    err, initial_state, reason = verify_input(initial_state, n)
    
    if isSolvable(n, initial_state) == False: 
        err = -2
        steps, frontierSize = None, None
        raise ValueError("This board is unsolvable (Error code = -2)")
    
    finish = PuzzleNode(n)
    p = PuzzleNode(n, current_state = initial_state)
    # initialize board with the given input 
    # print the initial board
    print "Initial board:"
    print p.__str__()
    print ""
    
    visited[p] = 0
    heuristic_value[p] = 0
    link[p] = None
    list_of_lists_solution = []
    
    if heuristic == manhattanDistance:
      print "Solving using Manhattan Distance..."
    elif heuristics == misplacedTiles:
      print "Solving using Misplaced Tiles..."
      
    print ""
     
    heapq.heappush(heap, (0, 0, p))
    # keep a count of the number of steps, and avoid an infinite loop
    for steps in xrange(1000000):
        f, junk, current = heapq.heappop(heap)
        if current == finish:
            print "Final board:"
            print current
            
            if prnt == True:
                print ""
                print "Solutions as lists of lists:"
                print list_of_lists_solution
                print ""
            runtime = time.time() - start_time
            return steps, frontierSize, err, runtime

        moves = current.get_moves()
        distance = visited[current]
        for mv in moves:
            if prnt == True:
                list_of_lists_solution.append(mv.current_state)

            # check for repeated states and avoid expanding them
            if mv not in visited or visited[mv] > distance + 1:
                visited[mv] = distance + 1
                if mv not in heuristic_value:
                    heuristic_value[mv] = heuristic(mv)
                link[mv] = current
                heapq.heappush(heap, (visited[mv] + heuristic_value[mv], -steps, mv))
        frontierSize = max(frontierSize,len(heap)) 
    else:
        raise Exception("Did not find a solution")

class PuzzleNode(object):
    def __init__(self, n, current_state=None):
        """Create an nxn block puzzle

        Use current_state to initialize to a specific state.
        """
        self.n = n
        self.n2 = n * n
        if current_state is None:
            self.current_state = [(x) % self.n2 for x in xrange(self.n2)]
        else:
            self.current_state = list(current_state)
        self.hsh = None
        self.last_move = []

    def __hash__(self):
        if self.hsh is None:
            self.hsh = hash(tuple(self.current_state))
        return self.hsh

    def __repr__(self):
        return "PuzzleNode(%d, %s)" % (self.n, self.current_state)

    def __str__(self):
        ys = ["%2d" % x for x in self.current_state]
        current_state = [" ".join(ys[steps:steps+self.n]) for steps in xrange(0,self.n2, self.n)]
        return "\n".join(current_state)

    def __eq__(self, other):
        return self.current_state == other.current_state

    def copy(self):
        return PuzzleNode(self.n, self.current_state)

    def get_moves(self):
        # Find the 0 tile, and then generate any moves we
        # can by sliding another block into its place.
        tile0 = self.current_state.index(0)
        def swap(i):
            j = tile0
            tmp = list(self.current_state)
            last_move = tmp[i]
            tmp[i], tmp[j] = tmp[j], tmp[i]
            result = PuzzleNode(self.n, tmp)
            result.last_move = last_move
            return result

        if tile0 - self.n >= 0:
            yield swap(tile0 - self.n)
        if tile0 + self.n < self.n2:
            yield swap(tile0 + self.n)
        if tile0 % self.n > 0:
            yield swap(tile0 - 1)
        if tile0 % self.n < self.n - 1:
            yield swap(tile0 + 1)
            
    def build_path(state, finish, parent):
        """
        Reconstruct the path from state to finish given
        a dict of parent links.

        """
        x = finish
        current_state = [x]
        while x != state:
            x = parent[x]
            current_state.append(x)
        current_state.reverse()
        return current_state
      
def verify_input(initial_state, n):
  err = 0
  reason = "Input is valid"
  
  if n < 2 or n >= 128:
    err = -1
    reason = "N is not between 2 and 128"
    
  if len(initial_state) != n ** 2:
    err = -1
    reason = "Board size is not N^2"
    
  sorted_initial_state = sorted(initial_state)
  valid_list = range(n**2)
  
  if sorted_initial_state != valid_list:
    err = -1
    reason = "Your range of numbers do not match the number of tiles"
    
    if err == -1:
      raise ValueError(reason)
  return err, initial_state, reason
  
def misplacedTiles(position):
    """Returns the number of tiles out of place."""
    n2 = position.n2
    c = 0
    for steps in xrange(n2):
        if position.current_state[steps] != steps:
            c += 1
    return c
  
def manhattanDistance(position):
    n2 = position.n2
    state = []
    for i in xrange(n2):
      state.append(position.current_state[i])
    index = [value for value in xrange(n2)]
    
    manhattan = 0
    idxx = []
    for var in xrange(n2):
        idx = state.index(var)
        idxx.append(idx)
        board_x = idx / 3
        board_y = idx % 3
        x = index[var] / 3
        y = index[var] % 3
        manhattan += math.fabs(x - board_x)
        manhattan += math.fabs(y - board_y)
    
        return manhattan
    
heuristics = [misplacedTiles, manhattanDistance]
  
def main(n, initial_state):
    # first, flatten the input board to a 1-D array
    initial_state = [item for sublist in initial_state for item in sublist]
    # solving p (n * n) to goal state using heuristic
    steps, frontier_size, err, runtime = solvePuzzle_solvable(n, initial_state, heuristics[1])
    print "Number of steps:", steps
    print "Frontier size:", frontier_size
    print "Error:", err
    print "Runtime: {}s".format(runtime)

n = 3
current_state = [[7, 0, 2], [8, 5, 3], [6, 4, 1]]
main(n, current_state)


Testing if board: [7, 0, 2, 8, 5, 3, 6, 4, 1] is solvable
Total inversions: 19


ValueError: ignored